In [0]:
storageAccount="cookbookadlsgen2storage1"
mountpoint = "/mnt/Gen2Source"
storageEndPoint ="abfss://rawdata@{}.dfs.core.windows.net/".format(storageAccount)
print ('Mount Point ='+mountpoint)

#ClientId, TenantId and Secret is for the Application(ADLSGen2App) was have created as part of this recipe
clientID =""
tenantID =""
clientSecret =""
oauth2Endpoint = "https://login.microsoftonline.com/{}/oauth2/token".format(tenantID)


configs = {"fs.azure.account.auth.type": "OAuth",
           "fs.azure.account.oauth.provider.type": "org.apache.hadoop.fs.azurebfs.oauth2.ClientCredsTokenProvider",
           "fs.azure.account.oauth2.client.id": clientID,
           "fs.azure.account.oauth2.client.secret": clientSecret,
           "fs.azure.account.oauth2.client.endpoint": oauth2Endpoint}

try:
  dbutils.fs.mount(
  source = storageEndPoint,
  mount_point = mountpoint,
  extra_configs = configs)
except e:
  print("Already Mounted")
    


In [0]:
from pyspark.sql.types import *
from pyspark.sql.functions import col
from pyspark.sql.functions import col,sum,avg,max

In [0]:
df_parquet = spark.read.format("parquet").option("header",True).load("/mnt/Gen2Source/Customer/parquetFiles")

In [0]:
display(df_parquet.show(10))

+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+
C_CUSTKEY| C_NAME| C_ADDRESS|C_NATIONKEY| C_PHONE|C_ACCTBAL|C_MKTSEGMENT| C_COMMENT|
+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+
 34136|Customer#000034136|UjBY ZyJnHesu0bxL...| 0|10-863-612-2949| 3397.87| BUILDING|es. furiously flu...|
 31731|Customer#000031731|Zq9KcCB7jMCIGMj32...| 0|10-250-546-6987| 8466.07| FURNITURE|counts lose quick...|
 33511|Customer#000033511| bz2D0HrwQiaz| 0|10-618-323-6263| 5331.22| AUTOMOBILE|ons are slyly ins...|
 45695|Customer#000045695|Oc2Yin5Gcqudb6RGE...| 0|10-414-601-9544| 5340.52| AUTOMOBILE|riously furious p...|
 36768|Customer#000036768| ,8Q8TZ48Wme| 0|10-950-600-3087| 9229.21| MACHINERY|ounts engage quic...|
 37449|Customer#000037449|kSezsusukDqm37pMN...| 0|10-231-238-1522| 1173.53| AUTOMOBILE|ding dependencies...|
 46970|Customer#000046970| uZbY1HQYjGobtk8ppkV| 0|10-361-617-6664| 8689.85| AUTOMOBILE|xes boost furious...|
 29109|Customer#000029109|FG6PVFkL69,LZWcl4...| 0|10-770-489-7404| 1939.54| AUTOMOBILE| above the furiou...|
 43753|Customer#000043753|xWtHn81QCu6kSjIkd...| 0|10-180-882-4254| 4975.15| AUTOMOBILE|sts-- idly final ...|
 42429|Customer#000042429|g6AgY9ecZcwMhWvq5...| 0|10-616-749-7989| 5345.32| BUILDING|ing to the slyly ...|
+---------+------------------+--------------------+-----------+---------------+---------+------------+--------------------+
only showing top 10 rows

In [0]:
for i in range(1,20):
  df_parquet.write.format("delta").mode("append").save("/mnt/Gen2Source/CustomerDelta")

In [0]:
#Creating CustomerDelta table
spark.sql("CREATE TABLE CustomerDelta USING DELTA LOCATION '/mnt/Gen2Source/CustomerDelta/'")

--------------------------------------------------------------------------- 
 AnalysisException Traceback (most recent call last)
 <command-845488711598998> in <module> 
 1 #Creating CustomerDelta table 
 ----> 2 spark . sql ( "CREATE TABLE CustomerDelta USING DELTA LOCATION '/mnt/Gen2Source/CustomerDelta/'" ) 

 /databricks/spark/python/pyspark/sql/session.py in sql (self, sqlQuery) 
 707 [ Row ( f1 = 1 , f2 = u'row1' ) , Row ( f1 = 2 , f2 = u'row2' ) , Row ( f1 = 3 , f2 = u'row3' ) ] 
 708 """
 --> 709 return DataFrame ( self . _jsparkSession . sql ( sqlQuery ) , self . _wrapped ) 
 710 
 711 @ since ( 2.0 ) 

 /databricks/spark/python/lib/py4j-0.10.9-src.zip/py4j/java_gateway.py in __call__ (self, *args) 
 1303 answer = self . gateway_client . send_command ( command ) 
 1304 return_value = get_return_value(
 -> 1305 answer, self.gateway_client, self.target_id, self.name)
 1306 
 1307 for temp_arg in temp_args : 

 /databricks/spark/python/pyspark/sql/utils.py in deco (*a, **kw) 
 131 # Hide where the exception came from that shows a non-Pythonic 
 132 # JVM exception message. 
 --> 133 raise_from ( converted ) 
 134 else : 
 135 raise 

 /databricks/spark/python/pyspark/sql/utils.py in raise_from (e) 

 AnalysisException : Table default.CustomerDelta already exists;

In [0]:
for i in range(1,20):
  df_parquet.write.format("delta").partitionBy("C_MKTSEGMENT").mode("append").save("/mnt/Gen2Source/CustomerDeltaPartition")

In [0]:
#Creating CustomerDeltaPartition table
spark.sql("CREATE TABLE CustomerDeltaPartition USING DELTA LOCATION '/mnt/Gen2Source/CustomerDeltaPartition/'")

Out[8]: DataFrame[]

In [0]:
%sql
-- Updating non partitioned table
-- Immediately go to the Notebook 6_4_1.ConcurrencyTest and run the first cell (DELETE FROM CustomerDelta WHERE C_MKTSEGMENT='FURNITURE')
UPDATE CustomerDelta SET C_NATIONKEY = 4 WHERE C_MKTSEGMENT='MACHINERY'

In [0]:
%sql
-- Updating non partitioned table
-- Immediately go to the Notebook 6_4_1.ConcurrencyTest and run the second cell (DELETE FROM CustomerDeltaPartition WHERE C_MKTSEGMENT='FURNITURE')
UPDATE CustomerDeltaPartition SET C_NATIONKEY = 4 WHERE C_MKTSEGMENT='MACHINERY'

In [0]:
%sql

ALTER TABLE CustomerDeltaPartition SET TBLPROPERTIES ('delta.isolationLevel' = 'Serializable')

In [0]:
# After running this cell, Immediately go to the Notebook 6_4_1.ConcurrencyTest and run the third cell (DELETE FROM CustomerDeltaPartition WHERE C_MKTSEGMENT='MACHINERY' AND C_CUSTKEY=1377)
for i in range(1,20):
  df_parquet.where(col("C_MKTSEGMENT")=="MACHINERY").write.format("delta").mode("append").partitionBy("C_MKTSEGMENT").save("/mnt/Gen2Source/CustomerDeltaPartition")


In [0]:
%sql
ALTER TABLE CustomerDeltaPartition SET TBLPROPERTIES ('delta.isolationLevel' = 'WriteSerializable')